# Reachy kinematics

In this notebook we will move Reachy's arms!

As explained in other notebooks, you can set target position to the joints if and only if the motors are in stiff mode. There are two ways to send a new target position: 

* **goal_position**: when this attribute is set, the joint will try to reach it as fast as it can and it can be pretty fast (up to 700 degrees per second). You can lower the *speed_limit* but we don't recommend to use this method as it gets really hard to follow complex trajectories and have precise timing. Even though you can change the motor speed, you have no control over the acceleration.  


* **goto()**: instead of playing with the goal_position you can use the *goto* method that we implemented. This method generates a trajectory between the present position and the goal position. This trajectory is then interpolated at a predefined frequency (100Hz) to compute all intermediary target position that should be followed before reaching the final goal position. Depending on the interpolation mode chosen, you can have a better control over speed and acceleration.

The two interpolation modes that we use consist in working either linearly or with the minjerk function.

<img src="images/kinematics/interpolation.png" alt="drawing" width="500"/>

Both trajectories start and finish at the same point but don't follow the same intermediate positions.
The minimum jerk will slowly accelerate at the begining and slowly decelerate at the end. This make the movements way more natural, you'll see this below.

Now let's make our robot move! You will make the same arm movement but in three different ways, using the *goal_position* and the *goto* method with both modes of interpolation.

You will put Reachy's right arm in the right-angled position, so place it on a table and make sure that there are no obstacles along its way.

In [47]:
from reachy_sdk import ReachySDK

from reachy_sdk.trajectory.interpolation import InterpolationMode

In [2]:
reachy = ReachySDK(host='localhost')

In [3]:
# Reachy's right arm joints are joints 8 to 15
for joint in reachy.joints[8:16]:
    print(joint.name)

r_shoulder_pitch
r_shoulder_roll
r_arm_yaw
r_elbow_pitch
r_forearm_yaw
r_wrist_pitch
r_wrist_roll
r_gripper


### goal_position

First, we will use set the *goal_psoition* on each joint to put the right arm in a right-agnled position. 

In [17]:
for joint in reachy.joints[8:16]:
    joint.compliant = False

# Define what the goal_position of each joint will be
right_angle_position = {
    'r_shoulder_pitch': 0,
    'r_shoulder_roll': 0,
    'r_arm_yaw': 0,
    'r_elbow_pitch': -90,
    'r_forearm_yaw': 0,
    'r_wrist_pitch': 0,
    'r_wrist_roll': 0,
    'r_gripper': 0,
}

#Set each goal_position
from operator import attrgetter

for name, goal in zip(right_angle_position.keys(), right_angle_position.values()):
    joint = attrgetter(name)(reachy)
    joint.goal_position = goal

A bit too fast, isn't it?

Now we will put the joints back to compliant mode, so put your hand below the arm to prevent the arm from falling on the table.

In [18]:
for joint in reachy.joints[8:16]:
    joint.compliant = True

### goto with linear interpolation

In [76]:
# First put the joints of the right arm in stiff mode
for joint in reachy.joints[8:16]:
    joint.compliant = False

In [77]:
reachy.goto(
    {getattr(reachy, joint.name): gp for joint, gp in zip(reachy.joints[8:16], higher_pos.values())},
duration=2.0,
interpolation_mode=InterpolationMode.LINEAR,
)

In [78]:
# First put the joints of the right arm in stiff mode
for joint in reachy.joints[8:16]:
    joint.compliant = True

### goto with minjerk interpolation

In [79]:
# First put the joints of the right arm in stiff mode
for joint in reachy.joints[8:16]:
    joint.compliant = False

In [80]:
reachy.goto(
    {getattr(reachy, joint.name): gp for joint, gp in zip(reachy.joints[8:16], higher_pos.values())},
duration=2.0,
interpolation_mode=InterpolationMode.MINIMUM_JERK
)

In [81]:
for joint in reachy.joints[8:16]:
    joint.compliant = True

In [68]:
higher_pos = {joint.name: joint.present_position for joint in reachy.joints[8:16]}

In [69]:
higher_pos

{'r_shoulder_pitch': -6.4818465341343385,
 'r_shoulder_roll': -0.24008811291725043,
 'r_arm_yaw': 1.6263735891470539,
 'r_elbow_pitch': -119.60439878257708,
 'r_forearm_yaw': 1.6129031754433194,
 'r_wrist_pitch': 31.076923591451138,
 'r_wrist_roll': 7.77126117159357,
 'r_gripper': -9.824047205884222}

In [13]:
reachy.r_arm.forward_kinematics()

array([[ 0.02274451, -0.34644187, -0.93779567,  0.30715529],
       [ 0.18541031,  0.92322709, -0.33656314, -0.01128561],
       [ 0.98239794, -0.16622202,  0.08523225, -0.16876404],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [23]:
goal_pos = reachy.r_arm.inverse_kinematics(reachy.r_arm.forward_kinematics())

In [27]:
for joint in reachy.joints[8:16]:
    joint.compliant = True

In [25]:
for joint in reachy.joints[8:16]:
    print(joint.name)

r_shoulder_pitch
r_shoulder_roll
r_arm_yaw
r_elbow_pitch
r_forearm_yaw
r_wrist_pitch
r_wrist_roll
r_gripper


In [23]:
for joint, gp in zip(reachy.joints[8:16], goal_pos):
    joint.goal_position = gp

In [25]:
reachy.goto(
    {getattr(reachy, joint.name): gp for joint, gp in zip(reachy.joints[8:16], goal_pos)},
    duration=2.0,
)

In [29]:
{getattr(reachy, joint.name): gp for joint, gp in zip(reachy.joints[8:16], goal_pos)}

{<Joint name="r_shoulder_pitch" pos="-4.99" mode="compliant">: -7.307031502758012,
 <Joint name="r_shoulder_roll" pos="-4.20" mode="compliant">: -4.21526265221073,
 <Joint name="r_arm_yaw" pos="4.79" mode="compliant">: 5.189068731020693,
 <Joint name="r_elbow_pitch" pos="-49.89" mode="compliant">: -84.01775158307635,
 <Joint name="r_forearm_yaw" pos="7.77" mode="compliant">: 8.293689313434687,
 <Joint name="r_wrist_pitch" pos="-39.52" mode="compliant">: 0.8513088274288462,
 <Joint name="r_wrist_roll" pos="0.73" mode="compliant">: 0.429521922579425}

## Forward/Inverse kinematics

Both forward and inverse kinematics are available on Reachy.

For those who don't kow what these are, the forward kinematics compute the 3D position and orientation of any joint in the arm whereas the inverse kinematics 

Really be aware of the robot frame. 

Cordinates are in meter. 

Inverse kinematics can send you anywhere. 

<img src="images/kinematics/arm_axis.png" alt="drawing" width="300"/>